In [1]:
import pandas as pd
import numpy as np

# 1. Load data

In [2]:
train = pd.read_csv('train_data.csv')
# train

In [3]:
test = pd.read_csv('test_data.csv')
# test

In [4]:
valid = pd.read_csv('validation_data.csv')
# valid[valid['rating'] == 1]

In [5]:
item = pd.read_csv('item_fea.csv')
# item

In [6]:
user = pd.read_csv('user_fea.csv')
# user

In [7]:
link = pd.read_csv('links.csv')
# link

In [8]:
sample = pd.read_csv('sample_solution_data.csv')
# sample

# 2. Cluster items

In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=200, random_state=0).fit(item)
item_cluster = kmeans.labels_
# item_cluster

# 2. Cluster users

In [10]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(user)
user_cluster = kmeans.labels_
# user_cluster

# 3. Weight items

In [11]:
item_dict = dict()
for i in train[train['user_id'] == 0]['item_id']:
    if item_cluster[i] in item_dict:            
        item_dict[item_cluster[i]] += 100
    else:
        item_dict[item_cluster[i]] = 100
item_dict

{8: 1000, 151: 1000, 101: 1000, 186: 1000, 49: 300}

# 4. Link other users

In [12]:
link_list = list()
for i in link[link['src'] == 0]['des']:
    link_list.append(i)
# link_list

# 5. Weight items

In [13]:
for j in link_list:
    if user_cluster[j] == user_cluster[0]:
        w = 10
    else:
        w = 1
    for i in train[train['user_id'] == j]['item_id']:
        if item_cluster[i] in item_dict:            
            item_dict[item_cluster[i]] += w
        else:
            item_dict[item_cluster[i]] = w  
item_dict

{8: 1005,
 151: 1018,
 101: 1018,
 186: 1007,
 49: 384,
 124: 4,
 120: 6,
 79: 11,
 159: 11,
 4: 6,
 68: 15,
 139: 33,
 75: 18,
 103: 18,
 168: 26,
 178: 12,
 52: 18,
 180: 18,
 39: 17,
 86: 13,
 153: 12,
 55: 23,
 108: 16,
 87: 12,
 41: 19,
 2: 17,
 133: 5,
 194: 2,
 106: 13,
 166: 8,
 48: 5,
 77: 9,
 192: 4,
 32: 7,
 94: 4,
 99: 9,
 20: 5,
 72: 4,
 105: 15,
 155: 8,
 12: 4,
 110: 119,
 116: 2,
 128: 5,
 7: 7,
 71: 3,
 182: 9,
 5: 6,
 62: 8,
 29: 14,
 100: 5,
 42: 2,
 131: 7,
 113: 8,
 14: 6,
 34: 4,
 91: 6,
 67: 9,
 123: 4,
 109: 137,
 137: 6,
 90: 9,
 16: 15,
 69: 8,
 114: 3,
 1: 11,
 181: 86,
 121: 6,
 167: 11,
 138: 7,
 164: 5,
 150: 9,
 176: 7,
 53: 4,
 117: 7,
 163: 6,
 170: 2,
 56: 9,
 165: 10,
 130: 20,
 83: 6,
 158: 5,
 0: 3,
 93: 8,
 46: 3,
 119: 6,
 45: 7,
 78: 3,
 84: 5,
 135: 5,
 92: 3,
 160: 4,
 37: 6,
 31: 8,
 43: 6,
 33: 2,
 175: 10,
 89: 1,
 27: 1,
 199: 5,
 81: 7,
 97: 9,
 191: 2,
 171: 8,
 169: 2,
 60: 9,
 76: 4,
 140: 3,
 172: 7,
 73: 4,
 112: 3,
 44: 3,
 98: 14,
 

# 6. Test data

In [14]:
recommender_dict = dict()
for i in test[test['user_id'] == 0]['item_id']:
    if item_cluster[i] in item_dict:
        recommender_dict[i] = item_dict[item_cluster[i]]
# recommender_dict

# 7. Sort data

In [15]:
sort_orders = sorted(recommender_dict.items(), key=lambda x: x[1], reverse=True)
# sort_orders

In [16]:
user_id_list = list()
item_id_list = list()

In [17]:
for i in sort_orders[:10]:
    user_id_list.append(0)
    item_id_list.append(i[0])

In [18]:
# user_id_list

In [19]:
# item_id_list

In [20]:
def validing(user_id):
    
    item_dict = dict()
    for i in train[train['user_id'] == user_id]['item_id']:
        if item_cluster[i] in item_dict:            
            item_dict[item_cluster[i]] += 100
        else:
            item_dict[item_cluster[i]] = 100
            
    link_list = list()
    for i in link[link['src'] == user_id]['des']:
        link_list.append(i)
        
    for j in link_list:
        if user_cluster[j] == user_cluster[user_id]:
            w = 10
        else:
            w = 1
        for i in train[train['user_id'] == j]['item_id']:
            if item_cluster[i] in item_dict:            
                item_dict[item_cluster[i]] += w
            else:
                item_dict[item_cluster[i]] = w

    recommender_dict = dict()
    for i in valid[valid['user_id'] == user_id]['item_id']:
        if item_cluster[i] in item_dict:
            recommender_dict[i] = item_dict[item_cluster[i]]
    
    sort_orders = sorted(recommender_dict.items(), key=lambda x: x[1], reverse=True)
    
    i = 0
    while len(sort_orders) < 10:
        sort_orders.append([valid[valid['user_id'] == user_id]['item_id'].tolist()[i]])
        i += 1
    
    tmp = list()
    for i in sort_orders[:10]:
        tmp.append(i[0])
        
    correct = int(valid[(valid['user_id'] == user_id) & (valid['rating'] == 1)]['item_id'])    
    if correct in tmp:        
        return 1
    else:
        return 0

In [21]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2000, random_state=0).fit(item)
item_cluster = kmeans.labels_

In [22]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1, random_state=0).fit(user)
user_cluster = kmeans.labels_

In [23]:
user_id_list = list()
item_id_list = list()
score = 0

In [24]:
for i in range(2239):
    score += validing(i)

In [25]:
accuracy = score / 2399
print(score)
print(accuracy)

1755
0.7315548145060442


# Predict Data

In [26]:
def predicting(user_id):
    
    item_dict = dict()
    for i in train[train['user_id'] == user_id]['item_id']:
        if item_cluster[i] in item_dict:            
            item_dict[item_cluster[i]] += 100
        else:
            item_dict[item_cluster[i]] = 100
            
    link_list = list()
    for i in link[link['src'] == user_id]['des']:
        link_list.append(i)
        
    for j in link_list:
        if user_cluster[j] == user_cluster[user_id]:
            w = 10
        else:
            w = 1
        for i in train[train['user_id'] == j]['item_id']:
            if item_cluster[i] in item_dict:            
                item_dict[item_cluster[i]] += w
            else:
                item_dict[item_cluster[i]] = w

    recommender_dict = dict()
    for i in test[test['user_id'] == user_id]['item_id']:
        if item_cluster[i] in item_dict:
            recommender_dict[i] = item_dict[item_cluster[i]]
    
    sort_orders = sorted(recommender_dict.items(), key=lambda x: x[1], reverse=True)
    
    i = 0
    while len(sort_orders) < 10:
        sort_orders.append([test[test['user_id'] == user_id]['item_id'].tolist()[i]])
        i += 1
        
    for i in sort_orders[:10]:
        user_id_list.append(user_id)
        item_id_list.append(i[0])

In [27]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2000, random_state=0).fit(item)
item_cluster = kmeans.labels_

In [28]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=1, random_state=0).fit(user)
user_cluster = kmeans.labels_

In [29]:
user_id_list = list()
item_id_list = list()

In [ ]:
for i in range(2239):
    predicting(i)

In [ ]:
df = pd.DataFrame(list(zip(user_id_list, item_id_list)), columns = ['user_id', 'item_id']) 

In [ ]:
df.head(50)

In [ ]:
df.to_csv('output.csv', index=False) # cluster 100 is better than 200